In [1]:
import dask.dataframe as dd

# assume your dataframe is named "df"
df = dd.read_csv('./imap-dataset-2.csv')

In [2]:
len(df)

10779175

In [5]:
# Filter the rows where the "command" column is not equal to "login"
mask = df['command'] != 'login'
df_filtered = df.loc[mask]

# Drop the filtered rows from the Dask DataFrame
df_dropped = df.loc[~mask]

# Compute the result as a pandas DataFrame
result = df_dropped.compute()

NameError: name 'df' is not defined

In [4]:
# Save the result to a CSV file
result.to_csv('./login-events.csv', index=False)

In [18]:
### START FROM HERE
import dask.dataframe as dd

result = dd.read_csv('./login-events.csv')

In [19]:
print(len(result))

2278031


In [7]:
'''
Possible errors
Login Denied
r="xm001 no login failed.";msg=logonfailed:logondenied;errmsg=logonfailed:logondenied

Active Directory Error
r="z no [error="microsoft.exchange.data.storage.wrongserverexception:the user and the mailbox are in different active directory sites. --> microsoft.mapi.mapiexceptionmailboxintransit:mapiexceptionmailboxintransit: detected site violation (hr=0x0, ec=1292)" authresult=0] login failed.";msg="user:legacydn: /o=tue/ou=exchange administrative group (fydibohf23spdlt)/cn=recipients/cn=b28157203cc74cd483ce854c95539131-healthmailbox1e2de6d, recipienttype: usermailbox,

Login Ok
r=ok;msg="proxy:exch19mbx-op01.campus.adz.pi:1993:ssl;proxysuccess";activitycontextdata=8696cf72-0aa5-4660-b683-ca2279c8acea
'''

'\nPossible errors\nLogin Denied\nr="xm001 no login failed.";msg=logonfailed:logondenied;errmsg=logonfailed:logondenied\n\nActive Directory Error\nr="z no [error="microsoft.exchange.data.storage.wrongserverexception:the user and the mailbox are in different active directory sites. --> microsoft.mapi.mapiexceptionmailboxintransit:mapiexceptionmailboxintransit: detected site violation (hr=0x0, ec=1292)" authresult=0] login failed.";msg="user:legacydn: /o=tue/ou=exchange administrative group (fydibohf23spdlt)/cn=recipients/cn=b28157203cc74cd483ce854c95539131-healthmailbox1e2de6d, recipienttype: usermailbox,\n\nLogin Ok\nr=ok;msg="proxy:exch19mbx-op01.campus.adz.pi:1993:ssl;proxysuccess";activitycontextdata=8696cf72-0aa5-4660-b683-ca2279c8acea\n'

In [8]:
head = result.head(10000)

In [6]:
result.sort_values('dateTime')

,Unnamed: 0,index,dateTime,sessionId,seqNumber,sIp,cIp,user,duration,rqsize,rpsize,command,parameters,context,puid,server,ip,port
npartitions=9,,,,,,,,,,,,,,,,,,
,int64,int64,object,object,int64,object,object,object,int64,int64,int64,object,object,object,float64,object,object,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
len(detected_users_set)

1087

In [21]:
class IpInfo:
    def __init__(self, timestamp):
        self.counter = 0
        self.start_timestamp = timestamp
        self.users = set()
    
    def __str__(self):
        user_n = len(self.users)
        return f"User Info: counter = {self.counter}, {user_n} users"
            
    def __repr__(self):
        user_n = len(self.users)
        return f"User Info: counter = {self.counter}, {user_n} users"
    
    def increase_counter(self):
        self.counter += 1
        return self.counter
        
    def reset_counter(self, timestamp):
        self.counter = 0
        self.start_timestamp = timestamp
        
    def get_counter(self):
        return self.counter
    
    def get_start_timestamp(self):
        return self.start_timestamp
    
    def set_start_timestamp(self, start_timestamp):
        self.start_timestamp = start_timestamp
        
    def set_end_timestamp(self, end_timestamp):
        self.end_timestamp = end_timestamp
        
    def get_timestamp_diff(self, end_timestamp):
        # print(f"Calculating difference between {self.start_timestamp} - {end_timestamp}")
        diff = end_timestamp - self.start_timestamp
        diff = diff.item().total_seconds()
        # print(f"Result is {diff} seconds")
        return diff
        
    def get_users(self):
        return self.users
    
    def add_user(self, user):
        self.users.add(user)
    
    def reset_users(self):
        self.users.clear()

In [3]:
## Get malicious IPs, this is our ground truth

malicious_ip_set = set()
total_ip_set = set()
malicious_ip_dic = {}

for index, row in result.iterrows():
    # Add new ip to hashmap
    ip = row['ip']
    context = row['context']
    total_ip_set.add(ip)
    if 'xm001' in context:
        if not malicious_ip_dic.get(ip):
            ip_info = IpInfo(None)
            malicious_ip_dic[ip] = ip_info
        else:
            ip_info = malicious_ip_dic.get(ip)
        user = row['user']
        ip_info.add_user(user)
        ip_info.increase_counter()
        malicious_ip_set.add(ip)
        

print(len(malicious_ip_set))
print(len(total_ip_set))

50794
51057


In [34]:
# Now make it for IP addresses instead of user
# For true positive rate look at the command number
# This is one of the first versions of the script

import numpy as np

ip_dic = {}
detected_ip_set = set()
successfull_login_attempt = set()
malicious_ips = set()

# Define our tolerance levels
# 3600

# 4, 10000 -> 0.9924714871622001
# 4, 3600 -> 0.9771856495994427
# 7, 3600 -> 0.6372549019607843
# 3, 3600 -> 0.9905811763064257
# 9, 3600 -> 0.3194444444444444
attempts_tolerance, seconds_tolerance = 9, 3600
    
# Loop through each row of dask dataframe
for index, row in result.iterrows():
    # Add new ip to hashmap
    ip = row['ip']
    datetime = row['dateTime']
    timestamp = np.datetime64(datetime)
    
    if not ip_dic.get(ip):
        # initialize object
        ip_info = IpInfo(timestamp)
        ip_dic[ip] = ip_info
        
    # Check if the "context" column contains the string "logondenied"
    # This way we are ignoring Active Directory Error errors
    if 'logondenied' in row['context']:
        
        ip_info = ip_dic[ip]
        ip_info.increase_counter()
        count = ip_info.get_counter()

        if ip_info.get_counter() >= attempts_tolerance:       
            diff = ip_info.get_timestamp_diff(timestamp)
            # x login attemps in less than y seconds
            if diff < seconds_tolerance:
                detected_ip_set.add(ip)
            else:
                ip_info.reset_counter(timestamp)
    elif 'ok' in row['context']:
        ip_info.reset_counter(timestamp)
        

print(f"detected_ip_set has size {len(detected_ip_set)}")

/tmp/ipykernel_42229/3946504615.py:29: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_42229/3946504615.py:29: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_42229/3946504615.py:29: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_42229/3946504615.py:29: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_42229/3946504615.py:29: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_42229/3946504615.py:29: DeprecationWarning: parsing timezone awar

detected_ip_set has size 5551


In [29]:
# Final Script

import numpy as np

# Treats the IP as /24 and returns the shortned result
def get_short_ip(ip):
    split_ip = ip.split('.')
    split_ip.pop()
    short_ip = '.'.join(split_ip)
    return short_ip

ip_dic = {}
detected_ip_set = set()
successfull_login_attempt = set()
malicious_ips = set()
malicious_users_set = set()
short_ips = set()

attempts_tolerance, seconds_tolerance = 3, 3600
    
# Loop through each row of dask dataframe
for index, row in result.iterrows():

    ip = row['ip']
    user = row['user']
    datetime = row['dateTime']
    timestamp = np.datetime64(datetime)
    
    if not ip_dic.get(ip):
        # initialize object
        ip_info = IpInfo(timestamp)
        ip_dic[ip] = ip_info
        
    context = row['context']    
    # Check if the "context" column contains the string "logondenied"
    # This way we are ignoring Active Directory Error errors
    if 'logondenied' in context:
        
        ip_info = ip_dic[ip]
        count = ip_info.increase_counter()
        diff = ip_info.get_timestamp_diff(timestamp)
        short_ip = get_short_ip(ip) # treats the IP as /24
        ip_info.add_user(user)
        
        if (ip_info.get_counter() >= attempts_tolerance \
                and diff < seconds_tolerance) \
                or user in malicious_users_set \
                or short_ip in short_ips:       
            detected_ip_set.add(ip)
            short_ips.add(short_ip)
            malicious_users_set = malicious_users_set.union(ip_info.get_users())
            
    elif 'ok' in context:
        # this can be added to show that an affected user was broken into
        if ip in detected_ip_set:
            successfull_login_attempt.add(user)
        # reset counters
        ip_info.reset_counter(timestamp)
        ip_info.reset_users()
        

print(f"detected_ip_set has size {len(detected_ip_set)}")

/tmp/ipykernel_51319/1802560390.py:27: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_51319/1802560390.py:27: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_51319/1802560390.py:27: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_51319/1802560390.py:27: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_51319/1802560390.py:27: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  timestamp = np.datetime64(datetime)
/tmp/ipykernel_51319/1802560390.py:27: DeprecationWarning: parsing timezone awar

detected_ip_set has size 41387


In [30]:
successfull_login_attempt

{'c.brolly@adz.pi', 'jincan'}

In [31]:
intersection = malicious_ip_set.intersection(detected_ip_set)
print(f"len of malicious_ip_set {len(malicious_ip_set)}")
print(f"len of detected ips {len(detected_ip_set)}")
print(f"len of intersection {len(intersection)}")
ratio = len(intersection) / len(detected_ip_set)
false_positive_rate = 1 - ratio
print(ratio)
print(f"false positive rate of {false_positive_rate}")

NameError: name 'malicious_ip_set' is not defined

In [ ]:
'''
Tarik 3, 3600
len of malicious_ip_set 50794
len of detected ips 41387
len of intersection 41231
0.9962307004614975
false positive rate of 0.0037692995385024686

Tarik++
len of malicious_ip_set 50794
len of detected ips 49992
len of intersection 49829 (163 false positives)
0.9967394783165306
false positive rate of 0.003260521683469353
'''

In [63]:
49992 - 49829

163

In [44]:
for cons in range(2, 10):
    low_attempts = 0
    for key, value in sorted(malicious_ip_dic.items()): # Note the () after items!
        if value.get_counter() < cons:
            low_attempts += 1
        # print(key, value)
    print(f"{low_attempts} IPs with less than {cons} consecutive failed login attempts")

13135 IPs with less than 2 consecutive failed login attempts
19334 IPs with less than 3 consecutive failed login attempts
23280 IPs with less than 4 consecutive failed login attempts
26156 IPs with less than 5 consecutive failed login attempts
28327 IPs with less than 6 consecutive failed login attempts
30096 IPs with less than 7 consecutive failed login attempts
31638 IPs with less than 8 consecutive failed login attempts
32935 IPs with less than 9 consecutive failed login attempts


In [6]:
import pandas as pd

ips = pd.read_csv('sorted-ip.csv')

In [16]:
ips.head(380)

,user,ip
0,t.cleaving@adz.pi,1201
1,e.o.met.sedan@adz.pi,1178
2,g.d.l.kerflop@adz.pi,1059
3,d.s.r.e.g.crow@adz.pi,943
4,s.axis@adz.pi,911
...,...,...
375,e.clang@adz.pi,200
376,g.t.g.gourmet@adz.pi,200
377,o.a.l.turnip@adz.pi,200
378,l.prospect@adz.pi,200
